In [15]:
import pandas as pd
import numpy as np
import urllib3
import matplotlib.pyplot as plt
import requests
from urllib.parse import urlencode
import time
%matplotlib inline

In [3]:
cubes = requests.get('https://municipaldata.treasury.gov.za/api/cubes/', allow_redirects=True).json()

In [4]:
data = pd.DataFrame(cubes['data'])

In [10]:
data.head(5)

,description,label,name,uri
0,Analysis of creditors aggregated by age of deb...,Aged Creditor Analysis,aged_creditor,https://municipaldata.treasury.gov.za/api/cube...
1,Details of grants received from national gover...,Conditional Grants,conditional_grants,https://municipaldata.treasury.gov.za/api/cube...
2,Statement of Financial Performance. How a muni...,Income and Expenditure,incexp,https://municipaldata.treasury.gov.za/api/cube...
3,Analysis of debtors by age of debt. Does not i...,Aged Debtor Analysis,aged_debtor,https://municipaldata.treasury.gov.za/api/cube...
4,"Purchase, repair and renewal of capital assets.",Capital Acquisition,capital,https://municipaldata.treasury.gov.za/api/cube...


In [ ]:
aged = requests.get(data.uri[0])

In [8]:
def url_change(x, url = 'https://municipaldata.treasury.gov.za/api/cubes/'):
    return ''.join([url,x.split(url)[1].replace('/','')])

In [9]:
data.uri= data.uri.apply(lambda x: url_change(x))

In [ ]:
test = requests.get(data.uri[7], allow_redirects=True).json()

In [55]:
data

,description,label,name,uri
0,Analysis of creditors aggregated by age of deb...,Aged Creditor Analysis,aged_creditor,https://municipaldata.treasury.gov.za/api/cube...
1,Details of grants received from national gover...,Conditional Grants,conditional_grants,https://municipaldata.treasury.gov.za/api/cube...
2,Statement of Financial Performance. How a muni...,Income and Expenditure,incexp,https://municipaldata.treasury.gov.za/api/cube...
3,Analysis of debtors by age of debt. Does not i...,Aged Debtor Analysis,aged_debtor,https://municipaldata.treasury.gov.za/api/cube...
4,"Purchase, repair and renewal of capital assets.",Capital Acquisition,capital,https://municipaldata.treasury.gov.za/api/cube...
5,Specific expenditure amounts from audited fina...,"Unauthorised, Irregular, Fruitless and Wastefu...",uifwexp,https://municipaldata.treasury.gov.za/api/cube...
6,The movement of cash into and out of the munic...,Cash Flow,cflow,https://municipaldata.treasury.gov.za/api/cube...
7,General information about the municipality inc...,Municipalities,municipalities,https://municipaldata.treasury.gov.za/api/cube...
8,Outcomes of financial statement audits from th...,Audit Opinions,audit_opinions,https://municipaldata.treasury.gov.za/api/cube...
9,Names and contact information for high level m...,Municipal Officials,officials,https://municipaldata.treasury.gov.za/api/cube...


In [ ]:
requests.get(test['endpoints'][2], allow_redirects=True).json()

In [16]:
args = urlencode({'municipality.code':"BUF"})

In [57]:
target_cashflow = 'https://municipaldata.treasury.gov.za/api/cubes/cflow/facts'

In [29]:
target_munis = 'https://municipaldata.treasury.gov.za/api/cubes//facts'

In [28]:
target_officials = 'https://municipaldata.treasury.gov.za/api/cubes/officials/facts'

In [37]:
munis = requests.get(target_munis,allow_redirects=True).json()['data']

In [27]:
officials = pd.DataFrame(requests.get(target_officials,allow_redirects=True).json()['data'])
cashflows = pd.DataFrame(requests.get(target_cashflow,allow_redirects=True).json()['data'])

In [68]:
municipalities = []

for muni in munis:
    temp = {}
    temp['municipality.demarcation_code'] = muni['municipality.demarcation_code']
    temp['name'] = muni['municipality.name']
    temp['province'] = muni['municipality.province_name']
    temp['category'] = muni['municipality.miif_category']
    temp['area'] = muni['area']
    municipalities.append(temp)
municipalities = pd.DataFrame(municipalities)

In [50]:
officials = officials['data']

In [53]:
officials = pd.DataFrame(officials)

In [69]:
officials.columns

Index(['contact_details.email_address', 'contact_details.fax_number',
       'contact_details.name', 'contact_details.phone_number',
       'contact_details.title', 'municipality.demarcation_code', 'role.role'],
      dtype='object')

In [72]:
municipalities.columns

Index(['area', 'category', 'municipality.demarcation_code', 'name',
       'province'],
      dtype='object')

In [75]:
pd.merge(municipalities, officials, how='inner', on='municipality.demarcation_code').groupby('contact_details.title').count()

,area,category,municipality.demarcation_code,name,province,contact_details.email_address,contact_details.fax_number,contact_details.name,contact_details.phone_number,role.role
contact_details.title,,,,,,,,,,
Adv,5,5,5,5,5,5,5,5,5,5
Cllr,169,169,169,169,169,157,169,169,169,169
Dr,13,13,13,13,13,13,13,13,13,13
Mr,789,789,789,789,789,779,773,788,784,789
Mrs,280,280,280,280,280,278,272,280,279,280
Ms,833,833,833,833,833,824,806,833,831,833
